In [38]:
import pandas as pd
import regex as re

In [39]:
# Import AOP EC table data
AOP_EC_table = pd.read_csv("../aop_ke_ec.csv")
# AOP_EC_table = pd.read_csv("../aop150_ke_ec.csv")

AOP_num = 6

#Set output file name
outfile = f"../aop{AOP_num}_from_script.ttl"
print(AOP_num)

6


In [ ]:
import os
def prepend_line(file_name, line):
    """ Insert given string as a new line at the beginning of a file """
    # define name of temporary dummy file
    dummy_file = file_name + '.bak'
    # open original file in read mode and dummy file in write mode
    with open(file_name, 'r') as read_obj, open(dummy_file, 'w') as write_obj:
        # Write given line to the dummy file
        write_obj.write(line + '\n')
        # Read lines from original file one by one and append them to the dummy file
        for line in read_obj:
            write_obj.write(line)
    # remove original file
    os.remove(file_name)
    # Rename dummy file as the original file
    os.rename(dummy_file, file_name)

In [40]:
# AOP150 = AOP_EC_table[AOP_EC_table["aop"] == "Aop:150"]
AOP = AOP_EC_table[AOP_EC_table["AOP"] == f"Aop:{AOP_num}"]
# AOP150 = AOP_EC_table[AOP_EC_table["aop"] == "Aop:23"]

In [41]:
# write header 
header = '''@prefix : <http://www.semanticweb.org/mmandal/ontologies/2022/4/untitled-ontology-76#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@base <http://www.w3.org/2002/07/owl#> .

[ rdf:type owl:Ontology ;
   owl:imports <http://purl.obolibrary.org/obo/go/releases/2022-01-13/go.owl> ,
               <http://purl.obolibrary.org/obo/ro/releases/2022-01-20/ro.owl>
 ] .
'''

with open(outfile, "w+") as f:
    f.write(header)

In [42]:
def process_term(input_id, name):
    """
    Takes a term and id and returns ttl class and instance statements
    """
    if ":" in input_id:
        input_id_str = f"{input_id[:2]}_{input_id[3:]}"
    else:
        input_id_str = input_id
    name = re.sub(" ", "_", name)
    class_statement = f'''\n###  http://purl.obolibrary.org/obo/{input_id_str}\n\t<http://purl.obolibrary.org/obo/{input_id_str}> rdf:type owl:Class .\n\n'''
    instance_statement = f'''###  http://www.co-ode.org/ontologies/ont.owl#{name}\n<http://www.co-ode.org/ontologies/ont.owl#{name}> rdf:type owl:NamedIndividual ,\n\t<http://purl.obolibrary.org/obo/{input_id_str}> '''

    return class_statement, instance_statement;

# def get_action_id(action):
#     """
#     Takes an action word and returns an RO relationship ID
#     simplified version
#     """
#     if action == "increased":
#         return "RO_0000057"
#     elif action == "decreased":
#         return "RO_0000058"
#     else:
#         return ""

def get_action_id (act,  source_1, id_1, term_1, source_2, id_2, term_2):
#     if act == "": # step action
#     else: # row action
        
    if act == "increased":
        return "RO_0000057"
    else:
        return "RO_0000058"

    

def add_action(act, source_1, id_1, term_1, source_2, id_2, term_2, outfile):
    action_id = get_action_id(act, source_1, id_1, term_1, source_2, id_2, term_2)
    row = f"{act},{action_id},{source_1},{id_1},{term_1},{source_2},{id_2},{term_2}\n"
    with open(outfile, "a") as f:
            f.write(row)
    if act != "":
        action_statement = f";\n\t<http://purl.obolibrary.org/obo/{action_id}> <http://www.co-ode.org/ontologies/ont.owl#{term_2}> "
        return(action_statement)
    else:
        return("")


In [43]:
classes = {}
instances = {}
steps = {}
# Cycle through rows and create classes and instances
outfile_csv = f"../outfile_09122022_AOP{AOP_num}.csv"
with open(outfile_csv, "w+") as f:
            f.write(f"action,action_id,source_1,id_1,term_1,source_2,id_2,term_2\n")

for index, row in AOP.iterrows():
    row = row.rename(lambda x: re.sub("[\s\/]", "_", x.lower()))
    
    # if object_id is not already in classes.keys() add object_id
    if row.object_id not in classes.keys() and not pd.isna(row.object_id):
        class_statement, instance_statement = process_term(row.object_id, row.object_term)
        classes[row.object_id] = class_statement
        instances[row.object_id] = instance_statement
        
    # if process_phenotype_id is not already in instances.keys() add process_phenotype_id
    if row.process_phenotype_id not in instances.keys() and not pd.isna(row.process_phenotype_id):
        class_statement, instance_statement = process_term(row.process_phenotype_id, row.process_phenotype_term)
        classes[row.process_phenotype_id] = class_statement
        instances[row.process_phenotype_id] = instance_statement
    
    row_action_statement = add_action(row.action, row.object_source, row.object_id ,row.object_term, row.process_phenotype_source, row.process_phenotype_id,
                                      row.process_phenotype_term, outfile_csv)
#     instances[row.process_phenotype_id] = instances[row.process_phenotype_id] + row_action_statement
#     if index != 0:
#         step_action_statement = add_action("", AOP150.iloc[index-1]['process_phenotype_source'], 
#                                            AOP150.iloc[index-1]['process_phenotype_id'], 
#                                            AOP150.iloc[index-1]['process_phenotype_term'], 
#                                            row.process_phenotype_source, row.process_phenotype_id, 
#                                            row.process_phenotype_term)
#         instances[row.process_phenotype_id] = instances[row.process_phenotype_id] + step_action_statement
    
    
    
          
with open(outfile, "a") as f:
            f.write("\n\n#################################################################")     
            f.write("\n#   Classes")  
            f.write("\n#################################################################\n\n")  
for c, s in classes.items():
    with open(outfile, "a") as f:
            f.write(s)
with open(outfile, "a") as f:
            f.write("\n\n#################################################################")     
            f.write("\n#   Instances")  
            f.write("\n#################################################################\n\n")  
for i, s in instances.items():
    with open(outfile, "a") as f:
            f.write(s + ".\n\n")


In [44]:
# print(AOP.iloc[0])
# import os
# print(os.getcwd())